# FedAvg with SAG (Scatter & Gather) workflow with Site-Specific Configurations

In this example, we will demonstrate the Feg Avg. SAG workflow using CIFAR10 dataset with site-specific configurations and deploy_map. 

We build on top of the previous example [FedAvg with SAG workflow](../sag/sag.ipynb#title)

Different from the previous examples, we like to demonstrate how to deploy different "app" to different sites using different configurations. 

The next section is going to be the same as previous example, please follow the steps 
  * [Understanding Fed Average and SAG](../sag/sag.ipynb#sag)
  * [Convert training code to federated learning training code](../sag/sag.ipynb#code)
  * [Prepare Data](../sag/sag.ipynb#data)


The next step is to preapre job folder and configurations. We will start from here. 

## Job Folder and Configurations

Before we dive into the details of the job configurations, let's first introduce few terminologies.

### Job and App

So far, when we referring NVFLARE job, we are referring the job folder which consists 

  * The job folder structure
  * The job configuration and 
  * The custom codes 

Actually, the complete picture of the job is the followings:

  * The job has special folder structure
  * The job is consists of many "apps" or "applications"
  * each application is consists app's custom code and configurations. 
  
  you probably has noticed the folder structure as 
```
  <job folder>
  | -- app
  | -----config/
  | --------config_fed_client.conf
  | --------config_fed_server.conf
  | -----custom
  | --------train.py
    
```
  In such case, we only defined one app. We could define many apps. 
  
  ```
  <job folder>
  | -- app_server
  | -- app_1
  | -- app_2
    
```
  in above,we defined three apps, each can be used at different sites. We will discuss how to map the app to different site using deploy_map

   
### deploy_map

What's "deploy_map" ? It is the "application to site deployment mapping". The deploy_map defines where to deploy these apps. 

For examaple, in meta.conf (or meta.json), you probably seen something like this: 

```
{
    "name": "my_job",
    "resource_spec": {},
    "deploy_map": {
        "app": [
            "@ALL"
        ]
    },
    "min_clients": 1,
    "mandatory_clients": [],
}


```
The deploy_map here
```
    "deploy_map": {
        "app": [
            "@ALL"
        ]
```
specify that the application "app" will be deployed to ALL sites. "@ALL" is special symbol, short-hand for all participanting clients and server. 


Here is another example of deploy_map

There are three "app"s: "app_server", "app_1", and "app_2". The deploy_map rules states, that "app_server" is only deploy to "server" site, "app_1" and "app_2" will be deployed to site-1 and site-2 respectively.

```
{
    "name": "cifar10_splitnn",
    "deploy_map": {
        "app_server": ["server"],
        "app_1": ["site-1"],
        "app_2": ["site-2"] 
    },
    "min_clients": 2,
}
```

Like in previous example, we are going to use an exisiting job template to create our job folder via Job CLI. You can refer the [Job CLI tutorials](../../../../tutorials/job_cli.ipynb).
  
The **Advanced Section** discussed the site-specific configuration as well. The tutorial is similar to this example. 
Now, lets create the job folder

## Create Job 

We are going to use Job CLI to create the job folder, so we can bring in the pre-defined job configuration from job templates. Let will examine what are available job templates


In [ ]:
! nvflare job list_templates

Looks like the sag_pt_deploy_map is the job template we need. Let's take the first look

In [ ]:
! nvflare job create \
-j /tmp/nvflare/sag_site_config -w sag_pt_deploy_map -force

Here we have three different apps : "app_server", "app_1" and "app_2". 
We like to change the followings: 

* change number of training rounds to 2
* change default app_script from "cifar10.py" to "train.py" for both app_1 and app_2
* change the app_1 batch_size to 4, app_2 batch_size to 6

In [ ]:
! nvflare job create \
-j /tmp/nvflare/sag_site_config -w sag_pt_deploy_map \
-f app_server/config_fed_server.conf num_rounds=2 \
-f app_1/config_fed_client.conf app_script=train.py app_config="--batch_size 4" \
-f app_2/config_fed_client.conf app_script=train.py app_config="--batch_size 6" \
-sd ../code/fl \
-force

>Note:
In the upcoming sections, we'll utilize the 'tree' command. To install this command on a Linux system, you can use the sudo apt install tree command. As an alternative to 'tree', you can use the ls -al command.

In [ ]:
! tree /tmp/nvflare/sag_site_config

We don't need the train.py in server

In [ ]:
! rm /tmp/nvflare/sag_site_config/app_server/custom/train.py

In [ ]:
! tree /tmp/nvflare/sag_site_config

## Run Job

Now we can run the job with simulator command.

In [ ]:
! nvflare simulator /tmp/nvflare/sag_site_config -t 2 -n 2 

The job should be running in the simulator mode. We are done with the training. 

In the next example [sag_model_learner](../sag_model_learner/sag_model_learner.ipynb), we will illustrate how to use the Model Learner API instead of the Client API,
and highlight why and when to use it.